<a href="https://colab.research.google.com/github/BiancadeFrancisco/SistemaRecomendacao_ItenKNN/blob/main/RECOMENDA%C3%87%C3%83O_ITEM_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-surprise

# BIBLIOTECA DO SCIKIT QUE JÁ TEM DIVERSOS ALGORITMOS DE RECOMENDAÇÃO

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=2811606 sha256=18c8fd6c31941b5823e827e6896d75a9c153c323b15d82e0119e48c90193ade3
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
import os
import numpy as np
import pandas as pd
from google.colab import files

## Arquivo de avaliações

Upload file `ratings.parquet`

In [ ]:
%%time
_ = files.upload()

Saving ratings.parquet to ratings.parquet
CPU times: user 935 ms, sys: 189 ms, total: 1.12 s
Wall time: 1min 22s


In [ ]:
df_ratings = pd.read_parquet('ratings.parquet')
df_ratings.tail()

,user_id,item_id,rating,timestamp
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648
1000208,6040,1097,4,956715569


## Arquivo de metadados dos itens

Upload file `movies.parquet`

In [ ]:
%%time
_ = files.upload()

Saving movies.parquet to movies.parquet
CPU times: user 83.1 ms, sys: 12.2 ms, total: 95.4 ms
Wall time: 7.71 s


In [ ]:
df_items = pd.read_parquet('movies.parquet')
df_items.set_index('item_id', inplace=True)
df_items.tail()

,title,genres
item_id,,
3948,Meet the Parents (2000),Comedy
3949,Requiem for a Dream (2000),Drama
3950,Tigerland (2000),Drama
3951,Two Family House (2000),Drama
3952,"Contender, The (2000)",Drama|Thriller


# Definindo os datasets de treino e validação

Como o ItemKNN é um modelo de parâmetros treináveis, podemos separar o dataset em treino e validação para observar se o treinamento está com uma boa generalização.

Dada a natureza sequencial do consumo de filmes, iremos utilizar o campo `timestamp` para fazer a quebra entre treino e validação: os primeiros `train_size` registros serão utilizados como treino e o restante como teste.

Além disso, a biblioteca `surprise` requer os seguintes nomes de colunas:

- `userID`: identificador do usuário
- `itemID`: identificador do item
- `rating`: _feedback_ do usuário

In [ ]:
df_ratings.tail()

,user_id,item_id,rating,timestamp
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648
1000208,6040,1097,4,956715569


In [ ]:
# CONSTRUIR UM DATASET DE TREINO E UM DATASET DE VALIDAÇÃO
# PARA ESSE CASO NÃO É UTILIZADO DADOS DE FORMA ALEATÓRIA, MAS SIM DE FORMA SEQUENCIAL

train_size = 0.8

# Ordenar por timestamp:
df_ratings = df_ratings.sort_values(by='timestamp', ascending=True)  # ORDENAR POR TEMPO

# Definindo train e valid sets:
df_train_set, df_valid_set = np.split(df_ratings, [ int(train_size*df_ratings.shape[0]) ])

# Redefinindo nomes para userID e itemID CFME A BIBLIOTECA SURPRISE EXIGE:
df_train_set = df_train_set.rename({'user_id': 'userID', 'item_id': 'itemID'}, axis=1)
df_valid_set = df_valid_set.rename({'user_id': 'userID', 'item_id': 'itemID'}, axis=1)

print ('Train size: ', df_train_set.shape)
print ('Valid size: ', df_valid_set.shape)

Train size:  (800167, 4)
Valid size:  (200042, 4)


In [ ]:
df_train_set

# ESSE DATASET POSSUI 80% DOS NOSSOS PRIMEIROS REGISTROS DE AVALIAÇÕES

,userID,itemID,rating,timestamp
1000138,6040,858,4,956703932
1000153,6040,2384,4,956703954
999873,6040,593,5,956703954
1000007,6040,1961,4,956703977
1000192,6040,2019,5,956703977
...,...,...,...,...
314102,1875,802,4,975768718
314151,1875,892,4,975768719
314073,1875,440,4,975768738
314225,1875,509,4,975768738


Algumas bibliotecas de sistemas de recomendação possuem classes específicas para trabalhar com os datasets. Desta forma, é preciso modificar os dados originais (em `pandas Dataframes` para as classes da biblioteca `surprise`).

In [ ]:
from surprise import Dataset, Reader #(READER E DATASET SÃO NECESSIDADES DA BIBLIOTECA SURPRISE)
def convert_train_valid_sets(df_train_set:pd.DataFrame, df_valid_set:pd.DataFrame):
  reader = Reader(rating_scale=(1, 5)) #PARA CONVETER DADOS DOS DATASETS DE TREINO E TESTE CFME EXIGENCIA DA BIBLIOTECA SURPRISE
  # The columns must correspond to user id, item id and ratings (in that order).
  train_set = (
      Dataset
      .load_from_df(df_train_set[['userID', 'itemID', 'rating']], reader)
      .build_full_trainset()
  )

  valid_set = (
      Dataset
      .load_from_df(df_valid_set[['userID', 'itemID', 'rating']], reader)
      .build_full_trainset()
      .build_testset()
  )

  return train_set, valid_set

train_set, valid_set = convert_train_valid_sets(df_train_set, df_valid_set)

In [ ]:
train_set

# Treinando o modelo

Para este exemplo prático iremos utilizar o **KNNWithMeans**, cuja fórmula de predição é dada abaixo:

$$\hat{r}_{ui} = b_i + \frac{\sum_{j \in N_u^k(i)} sim(i,j) \cdot (r_{uj} - b_j)}{\sum_{j \in N_u^k(i)} sim(i,j)}$$

Os hiperparâmetros escolhidos serão os indicados pela documentação para o dataset do MovieLens. No entanto, para outros datasets é preciso que haja uma busca de hiperparâmetros que seja ideal para os dados em questão. Para ler a definição dos hiperparâmetros, consulte a [documentação](https://surprise.readthedocs.io/en/stable/knn_inspired.html#surprise.prediction_algorithms.knns.KNNWithMeans).

**Nota**: para ler mais sobre testagem de hiperparâmetros, consulte a biblioteca [HyperOpt](http://hyperopt.github.io/hyperopt/).

In [ ]:
from surprise import KNNWithMeans  # KNNWithMeans = algoritmo de recomendação da biblioteca surprise

sim_options = {
    "name": "pearson_baseline",
    "user_based": False,  # compute similarities between items
}
model = KNNWithMeans(k=40, sim_options=sim_options, verbose=True)
model

# BIBLIOTECA HYPEROPT = para verificar parâmetros de otimização

Treinando o ItemKNN

In [ ]:
%%time
model.fit(train_set)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
CPU times: user 16 s, sys: 437 ms, total: 16.4 s
Wall time: 16.4 s


# Avaliando o treinamento


In [ ]:
user_id = df_valid_set['userID'].sample().tolist()[0]
item_id = df_valid_set['itemID'].sample().tolist()[0]

model.predict(uid=user_id, iid=item_id)

Prediction(uid=5878, iid=924, r_ui=None, est=4.341925616241596, details={'actual_k': 40, 'was_impossible': False})

## Predição do conjunto de validação

In [ ]:
%%time
df_valid_set['prediction'] = df_valid_set.apply(
    lambda x: model.predict(uid=x['userID'], iid=x['itemID']).est,
    axis=1
)

CPU times: user 22.7 s, sys: 47.5 ms, total: 22.8 s
Wall time: 23 s


In [ ]:
df_valid_set.tail()

,userID,itemID,rating,timestamp,prediction
825793,4958,2399,1,1046454338,2.561691
825438,4958,1407,5,1046454443,3.552168
825724,4958,3264,4,1046454548,3.270295
825731,4958,2634,3,1046454548,2.754026
825603,4958,1924,4,1046454590,2.584696


# Gerando Recomendações

In [ ]:
def recommend_n_items(model, user_id, item_ids:np.array, n=20):
  df_predictions = pd.DataFrame(columns=['item_id', 'score'])
  for item_id in item_ids:
    prediction = model.predict(uid=user_id, iid=item_id).est
    df_predictions.loc[df_predictions.shape[0]] = [item_id, prediction]

  user_predictions = (
      df_predictions
      .sort_values(by='score', ascending=False)
      .head(n)
      .set_index('item_id')
  )
  return user_predictions

user_id = 1875
recommendable_items = df_items.index.values
recommend_n_items(model, user_id, recommendable_items, n=5)

,score
item_id,
1830.0,5.0
989.0,5.0
3522.0,5.0
3881.0,5.0
1471.0,5.0


# K vizinhos próximos

Como o ItemKNN é um algoritmo baseado em vizinhança, podemos analisar quem são os K vizinhos mais próximos de um item-alvo.

In [ ]:
item_id = 1     # Toy Story
item_id = 1356  # Star Trek: First Contact
item_id = 260   # Star Wars: Episode IV - A New Hope
# item_id = 3578  # Gladiator

def get_item_k_neighbors(model, item_id, k=10):  # get_item_k_neighbors = FUNÇÃO DA BIBLIOTECA SURPRISE PARA MOSTRAR OS K PRÓXIMOS
  iid = model.trainset.to_inner_iid(item_id)
  neighbor_iids = model.get_neighbors(iid, k)
  item_ids = [model.trainset.to_raw_iid(iid) for iid in neighbor_iids]
  return item_ids

k = 10
title = df_items.loc[item_id]['title']
print (f'{k} vizinhos mais próximos de "{title}" (ID = {item_id})')
item_ids = get_item_k_neighbors(model, item_id, k)
df_items[df_items.index.isin(item_ids)]

10 vizinhos mais próximos de "Star Wars: Episode IV - A New Hope (1977)" (ID = 260)


,title,genres
item_id,,
1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
1198,Raiders of the Lost Ark (1981),Action|Adventure
1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
1291,Indiana Jones and the Last Crusade (1989),Action|Adventure
2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller
2628,Star Wars: Episode I - The Phantom Menace (1999),Action|Adventure|Fantasy|Sci-Fi
2640,Superman (1978),Action|Adventure|Sci-Fi
2716,Ghostbusters (1984),Comedy|Horror
3507,"Odd Couple, The (1968)",Comedy
